In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers import Dense,Bidirectional,GRU,Dropout,Input, Embedding,TextVectorization, LSTM
from tensorflow.keras.models import Model

In [ ]:
!wget https://www.manythings.org/anki/fra-eng.zip

In [ ]:
!unzip '/content/fra-eng.zip' -d '/content/dataset'

In [ ]:
text_dataset = tf.data.TextLineDataset('/content/dataset/fra.txt')

In [ ]:
for i in text_dataset.take(3):
  print(i)

In [ ]:
VOCAB_SIZE=20000
ENGLISH_SEQUENCE_LENGTH=64
FRENCH_SEQUENCE_LENGTH=64
EMBEDDING_DIM=300
BATCH_SIZE=64

In [ ]:
english_vectorize_layer=TextVectorization(
    standardize='lower_and_strip_punctuation',
    max_tokens=VOCAB_SIZE,
    output_mode='int',
    output_sequence_length=ENGLISH_SEQUENCE_LENGTH
)

In [ ]:
french_vectorize_layer=TextVectorization(
    standardize='lower_and_strip_punctuation',
    max_tokens=VOCAB_SIZE,
    output_mode='int',
    output_sequence_length=FRENCH_SEQUENCE_LENGTH
)

In [ ]:
def selector(input_text):
  split_text=tf.strings.split(input_text,'\t')
  return {'input_1':split_text[0:1],'input_2':'starttoken '+split_text[1:2]},split_text[1:2]+' endtoken'

In [ ]:
split_dataset=text_dataset.map(selector)

In [ ]:
for i in split_dataset.take(3):
  print(i)

In [ ]:
def separator(input_text):
  split_text=tf.strings.split(input_text,'\t')
  return split_text[0:1],'starttoken '+split_text[1:2]+' endtoken'

In [ ]:
init_dataset=text_dataset.map(separator)

In [ ]:
for i in init_dataset.take(3):
  print(i)

In [ ]:
english_training_data=init_dataset.map(lambda x,y:x)
english_vectorize_layer.adapt(english_training_data)

In [ ]:
french_training_data=init_dataset.map(lambda x,y:y)
french_vectorize_layer.adapt(french_training_data)

In [ ]:
def vectorizer(inputs,output):
  return {'input_1':english_vectorize_layer(inputs['input_1']),
          'input_2':french_vectorize_layer(inputs['input_2'])},french_vectorize_layer(output)

In [ ]:
dataset=split_dataset.map(vectorizer)

In [ ]:
for i in dataset.take(1):
  print(i)

In [ ]:
dataset=dataset.shuffle(2048).unbatch().batch(BATCH_SIZE).prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
NUM_BATCHES=int(200000/BATCH_SIZE)

In [ ]:
train_dataset=dataset.take(int(0.9*NUM_BATCHES))
val_dataset=dataset.skip(int(0.9*NUM_BATCHES))

In [ ]:
NUM_UNITS = 256

In [ ]:
input = Input(shape=(ENGLISH_SEQUENCE_LENGTH,), dtype="int64", name="input_1")
x=Embedding(VOCAB_SIZE, EMBEDDING_DIM, )(input)
encoded_input=Bidirectional(GRU(NUM_UNITS), )(x)

shifted_target=Input(shape=(FRENCH_SEQUENCE_LENGTH,), dtype="int64", name="input_2")
x=Embedding(VOCAB_SIZE,EMBEDDING_DIM,)(shifted_target)
x = GRU(NUM_UNITS*2, return_sequences=True)(x, initial_state=encoded_input)

x = Dropout(0.5)(x)
target=Dense(VOCAB_SIZE,activation="softmax")(x)
seq2seq_gru=Model([input,shifted_target],target)
seq2seq_gru.summary()

In [ ]:
class BLEU(tf.keras.metrics.Metric):
  def __init__(self, name='bleu_score'):
    super(BLEU, self).__init__()
    self.bleu_score = 0

  def update_state(self, y_true, y_pred, sample_weight=None):
    y_pred = tf.argmax(y_pred, axis=-1)
    self.bleu_score = 0
    for i,j in zip(y_pred, y_true):
      tf.autograph.experimental.set_loop_options() # add this line beacuse after removing element from j it give error.
      total_word = tf.math.count_nonzero(i)
      total_matches = 0
      for word in i:
        if word==0:
          break
        for q in range(len(j)):
          if j[q] == 0:
            break
          if j[q] == word:
            total_matches += 1
            j = tf.boolean_mask(j, [False if y == q else True for y in range(len(j))])
            break
    self.bleu_score += total_matches/total_word

  def result(self):
    return self.bleu_score/BATCH_SIZE

In [ ]:
seq2seq_gru.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(5e-4), metrics=[BLEU()])

In [ ]:
history=seq2seq_gru.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=15,)

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model_loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])

plt.title('model_accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
index_to_word={x:y for x, y in zip(range(len(french_vectorize_layer.get_vocabulary())),
                                   french_vectorize_layer.get_vocabulary())}

In [ ]:
def translator(english_sentence):
  tokenized_english_sentence=english_vectorize_layer([english_sentence])
  shifted_target = 'starttoken'

  for i in range(FRENCH_SEQUENCE_LENGTH):
    tokenized_shifted_target=french_vectorize_layer([shifted_target])

    output = seq2seq_gru.predict([tokenized_english_sentence,tokenized_shifted_target])
    french_word_imdex = tf.argmax(output, axis=-1)[0][i].numpy()
    current_word = index_to_word[french_word_imdex]

    if current_word == 'endtoken':
      break

    shifted_target += ' ' + current_word

  return shifted_target[11:]

In [ ]:
translator('What makes you think that is not true?')

In [ ]:
class Encoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, units):
    super(Encoder, self).__init__()
    self.embedding_dim = embedding_dim
    self.units = units
    self.vocab_size = vocab_size

  def build(self, input_shape):
    self.embedding = Embedding(self.vocab_size, self.embedding_dim)
    self.lstm = LSTM(self.units, return_sequences=True)

  def call(self, x):
    x = self.embedding(x)
    output = self.lstm(x)
    return output

In [ ]:
HIDDEN_UNITS = 256
EMBEDDING_DIM = 256
encoder = Encoder(VOCAB_SIZE, EMBEDDING_DIM, HIDDEN_UNITS)
encoder_output = encoder(tf.zeros((128, 64)))
encoder_output.shape

In [ ]:
class BahdanauAttention(tf.keras.Model):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.units = units

  def build(self, input_shape):
    self.w_1 = Dense(self.units)
    self.w_2 = Dense(self.units)
    self.w = Dense(1)

  def call(self, prev_dec_state, enc_state):
    scores = self.w(
        tf.nn.tanh(
            self.w_1(tf.expand_dims(prev_dec_state, -2)) + self.w_2(enc_state)
        ))
    attention_weights = tf.nn.softmax(scores, axis=1)
    context_vector = attention_weights*enc_state
    context_vector = tf.reduce_sum(context_vector, axis=1)
    return context_vector

In [ ]:
bahdanau_attention = BahdanauAttention(256)
context_vector = bahdanau_attention(tf.zeros((128, 32)), tf.zeros((128, 64, 32)))
context_vector.shape

In [ ]:
class Decoder(tf.keras.Model):
  def __init__(self,vocab_size,embedding_dim,dec_units,sequence_length):
    super(Decoder,self).__init__()
    self.embedding_dim=embedding_dim
    self.vocab_size=vocab_size
    self.dec_units=dec_units
    self.sequence_length=sequence_length

  def build(self,input_shape):
    self.dense=Dense(self.vocab_size,activation="softmax")
    self.gru=GRU(
        self.dec_units,return_sequences=True,return_state=True)
    self.attention=BahdanauAttention(self.dec_units)
    self.embedding=Embedding(self.vocab_size,self.embedding_dim)

  def call(self,x,hidden,shifted_target):
    outputs=[]
    context_vectors=[]
    shifted_target=self.embedding(shifted_target)

    for t in range(0,self.sequence_length):
      context_vector=self.attention(hidden,x)
      dec_input=context_vector+shifted_target[:,t]
      output,hidden=self.gru(tf.expand_dims(dec_input,1))
      outputs.append(output[:,0])

    outputs=tf.convert_to_tensor(outputs)
    outputs=tf.transpose(outputs, perm=[1,0,2])

    outputs=self.dense(outputs)
    return outputs

In [ ]:
decoder=Decoder(VOCAB_SIZE,EMBEDDING_DIM,HIDDEN_UNITS,FRENCH_SEQUENCE_LENGTH)
outputs=decoder(encoder_output,tf.zeros([128,HIDDEN_UNITS]),tf.zeros([128,64]))
outputs.shape

In [ ]:
### ENCODER
input = Input(shape=(ENGLISH_SEQUENCE_LENGTH,), dtype="int64", name="input_1")
encoder=Encoder(VOCAB_SIZE,EMBEDDING_DIM,HIDDEN_UNITS)
encoder_output=encoder(input)

### DECODER
shifted_target=Input(shape=(FRENCH_SEQUENCE_LENGTH,), dtype="int64", name="input_2")
decoder=Decoder(VOCAB_SIZE,EMBEDDING_DIM,HIDDEN_UNITS,FRENCH_SEQUENCE_LENGTH)
decoder_output,attention_weightss=decoder(encoder_output,tf.zeros([1,HIDDEN_UNITS]),shifted_target)

### OUTPUT
bahdanau=Model([input,shifted_target],decoder_output)
bahdanau.summary()

In [ ]:
bahdanau.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(5e-4),)

In [ ]:
history=bahdanau.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=30,)